# Dask Experiment


In [1]:
!pip install --quiet dask_kubernetes gql openpyxl pandas tqdm dateparser aiohttp pyproj geopandas matplotlib geojson "bokeh<3"

In [1]:
import os
!curl -O https://get.helm.sh/helm-v3.11.0-linux-amd64.tar.gz
!tar -xzf helm-v3.11.0-linux-amd64.tar.gz
!chmod +x ./linux-amd64/helm
!curl -LO "https://dl.k8s.io/release/$(curl -L -s https://dl.k8s.io/release/stable.txt)/bin/linux/amd64/kubectl"
!mv kubectl linux-amd64
!chmod +x ./linux-amd64/kubectl
os.environ["PATH"] += ":" + os.curdir + "/linux-amd64"
os.environ["KUBECONFIG"] = "coral_mapping_token.yaml"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 14.3M  100 14.3M    0     0  1289k      0  0:00:11  0:00:11 --:--:-- 1114k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   138  100   138    0     0    459      0 --:--:-- --:--:-- --:--:--   461
100 45.7M  100 45.7M    0     0  1727k      0  0:00:27  0:00:27 --:--:-- 1932k


In [2]:
from dask.distributed import Client
import geopandas as gpd
import enum
import re
from gql.client import TransportQueryError
from functools import partial#
import importlib
import dask
import csv

In [3]:
import arches_client
arches_client = importlib.reload(arches_client)
camel = arches_client.camel
studly = arches_client.studly

In [4]:
from dask.distributed import LocalCluster

In [5]:
#ENDPOINT = "http://fat-crl-dev-archesproject-api.fat-crl-dev.svc.cluster.local/"
ENDPOINT = "http://localhost:8088/"

In [6]:
skip = [
    #"area_name",
    #"monument_type",
    #"cultural_period",
    #"county",
    # "date_of_authorization",
    # "designation_start_date",
    #"feature_shape",
    #"named_location",
    "national_grid_reference",
    #"resourceid",
    #"primary_reference_number",
    #"shine_significance",
    #"spatial_accuracy_qualifier",
    #"ownership",
    #"geospatial_coordinates",
    # "compilation_start_date",#
    "actor",
    "images",
     #"display_date",
     #"associated_actor_start_date",
     #"associated_actor_display_date",
     #"risk_status",
]

In [7]:
import shapely.geometry, shapely.ops
from shapely import wkt
import pandas as pd
from pyproj import CRS, Transformer
igr = CRS('epsg:29901')
wgs84 = CRS('epsg:4326')
project = Transformer.from_crs(igr, wgs84, always_xy=True).transform
geopoint = wkt.loads("POINT (356384.3924000002 380874.0705999993)")
destination = shapely.ops.transform(project, geopoint)
type(destination)

shapely.geometry.point.Point

In [8]:
type_to_type = {
    "str": "String",
    "date": "String",
    "datetime": "String",
    "area_name": "AreaName",
    "risk_status": "RiskStatus",
    "confidence_of_dating": "ConfidenceOfDating",
    "monument_type": "[MonumentType]",
    "[area_name]": "[AreaName]",
    "[area_type]": "[AreaType]",
    "location": "JSONString"
}

In [9]:
fish_df = pd.read_csv("lookup_gentype_fish_poor.csv")
poor_fish = {row["GenTypeDescrip"].lower(): row["FISH"] for _, row in fish_df.iterrows()}
poor_fish

{'a.p. site': 'Campsite',
 'air raid shelter': 'Air Raid Shelter',
 'airfield': 'Airfield',
 'alignment': 'Pit Alignment',
 'alignment complex': 'Stone Alignment',
 'altar': 'Altar',
 'animal shelter': 'Animal Shed',
 'anti-aircraft battery': 'Anti Aircraft Battery',
 'anti-aircraft operations room': 'Anti Aircraft Operations Room',
 'arc of stones': 'Coffin Stone',
 'archaeologically sensitive': 'Archaeological Feature',
 'architectural fragment': 'Architectural Fragment',
 'assembly site': 'Assembly Rooms',
 'axe factory': 'Cable Factory',
 'bailey': 'Bailey',
 'ballast dump': 'Ballast Pond',
 'bank': 'Tank',
 'barrow': 'Barrow',
 'barrow cemetery': 'Barrow Cemetery',
 'bath house': 'Bath House',
 'battle site': 'Battlefield',
 'bier stone': 'Milestone',
 'boat house': 'Boat House',
 'boom': 'Boom',
 'boulder': 'Border',
 'bridge': 'Bridge',
 'bullaun': 'Bullring',
 'burial': 'Burial',
 'burial complex': 'Tile Burial',
 'burial ground': 'Friends Burial Ground',
 'burial mound': 'Burn

In [10]:
ConceptClient = arches_client.ConceptClient
concept_client = ConceptClient(ENDPOINT)
concept_client.connect()
concept = await concept_client.get_concept('acb5feaf-e491-4e53-bff2-2801167fd65a')
await concept_client.get_terms(concept["id"])

new_terms = [
    ["Barony"]
]
for term_set in new_terms:
    try:
        await concept_client.add_term("AdministrativeAreaName", "AdministrativeAreaName》" + "》".join(term_set))
    except TransportQueryError as e:
        pass
    if len(term_set) == 1:
        try:
            await concept_client.add_term("AdministrativeAreaType", "AdministrativeAreaType》" + "》".join(term_set))
        except TransportQueryError as e:
            pass


In [11]:
ConceptClient = arches_client.ConceptClient
concept_client = ConceptClient(ENDPOINT)
concept_client.connect()
concept = await concept_client.get_concept('acb5feaf-e491-4e53-bff2-2801167fd65a')
available_names = {term["label"].lower().strip(): term["label"] for term in await concept_client.get_terms(concept["id"])}

In [12]:
available_names

{'barony': 'Barony',
 'county/unitary authority': 'County/Unitary Authority',
 'essex': 'Essex',
 'greater london authority': 'Greater London Authority',
 'hertfordshire': 'Hertfordshire',
 'kent': 'Kent',
 'middlesex': 'Middlesex',
 'surrey': 'Surrey',
 'district/borough': 'District/borough',
 'broxbourne': 'Broxbourne',
 'elmbridge': 'Elmbridge',
 'epping forest': 'Epping Forest',
 'epsom and ewell': 'Epsom and Ewell',
 'reigate and banstead': 'Reigate and Banstead',
 'slough': 'Slough',
 'tandridge': 'Tandridge',
 'three rivers': 'Three Rivers',
 'thurrock': 'Thurrock',
 'london borough': 'London Borough',
 'barking and dagenham': 'Barking and Dagenham',
 'barnet': 'Barnet',
 'bexley': 'Bexley',
 'brent': 'Brent',
 'bromley': 'Bromley',
 'camden': 'Camden',
 'city of london': 'City of London',
 'city of westminster': 'City of Westminster',
 'croydon': 'Croydon',
 'ealing': 'Ealing',
 'enfield': 'Enfield',
 'greenwich': 'Greenwich',
 'hackney': 'Hackney',
 'hammersmith and fulham': '

In [13]:
import slugify

In [14]:
import dateparser
import geojson

def shape_to_geojson(shape):
    if not shape or pd.isna(shape):
        return None
    destination = shapely.ops.transform(project, shape)
    transformed = geojson.FeatureCollection(
        [geojson.Feature(geometry=destination)]
    )
    return geojson.dumps(transformed)

def naive_date_to_date_string(date):
    return dateparser.parse(date).strftime("%Y-%m-%d")
def naive_date_to_isoformat(date):
    return dateparser.parse(date).isoformat()

class RiskStatus(enum.Enum):
    NotAtRisk = "NotAtRisk"
    AtRisk = "AtRisk"
    
class Confidence(enum.Enum):
    Certain = "Certain"
    Possible = "Possible"
    Probable = "Probable"
    Uncertain = "Uncertain"
    
def concept_lookup(field, value):
    if field == "risk_status":
        return RiskStatus.NotAtRisk.name if pd.isna(value) or not value else RiskStatus.AtRisk.name
    elif field == "confidence_of_dating":
        return Confidence.Possible.name if value == "Possible" else Confidence.Certain.name
    elif field == "area_name":
        return "Antrim"
    
def monument_name(row):
    row = {k: v if pd.notna(v) else None for k, v in row.items()}
    loc = [f'{row.get("SiteName", None) or row.get("EditedType", None) or "(unknown)"}', row.get("Parish", None) or row.get("AllTownlands", None)]
    return ", ".join(l for l in loc if l)

def monument_type(row):
    if pd.isna(row['GenType']):
        return []
    fish = poor_fish[row['GenType'].lower()].replace("&", "and")
    return f"{studly(slugify.slugify(fish))}"

AREA_MAP = {
    "Barony": "Barony",
    "Parish": "Parish",
    "AllTownlands": "Townland",
    "Council": "Council",
    "Town": "Town"
}
AREA_NAME_SWAP = {
    "causeway coast and glens": "causeway coast & glens"
}
def localities(row):
    areas = []
    for col, typ in AREA_MAP.items():
        if pd.notna(name := row.get(col, None)) and AREA_NAME_SWAP.get(name.lower(), name.lower()) in available_names:
            name = AREA_NAME_SWAP.get(name.lower(), name.lower())
            name = available_names[name.lower()]
            areas.append({
                "areaName": studly(slugify.slugify(name.replace("&", "and"))),
                "areaType": typ
            })
    return areas
    
REFERENCE_NUMBERS = {
    "primaryReferenceNumber": "MonID",
    "legacyId": "InternalorPublic",
    "resourceid": "SMR_new"
}
def system_reference_numbers(row):
    row["InternalorPublic"] = f"{row['_batch']}:{row['MonID']}"
    return {
        k: row[v]
        for k, v in REFERENCE_NUMBERS.items()
         if v in row and pd.notna(row[v])
    }
    
# e.g. 0.0 => concept_lookup("risk_status", 0) => "NotAtRisk"
computed_fields = {
    "monument_name": (monument_name, "str"),
    "monument_type": (monument_type, "[MonumentType]"),
    "localities": (localities, "[MonumentLocalitiesInput]"),
    "system_reference_numbers": (system_reference_numbers, "[MonumentSystemReferenceNumbersInput]"),
}

In [15]:
types = dict(**__builtins__.__dict__)
types.update({
    "date": naive_date_to_date_string,
    "datetime": naive_date_to_isoformat,
    "location": shape_to_geojson,
})

mapping = {}
mapping_types_standardization = {
    "text": "str",
    "dropdown": "concept",
    "location": "location",
    "option select": "concept",
    "[text]": "[str]"
}
fields_to_coalesce = {}
def _coalesce_to_array(row, target, cb):
    return [cb(row[field]) for field in fields_to_coalesce[target] if pd.notna(row[field])]

with open("mapping_file_new.csv", "r") as fd:
    reader = csv.DictReader(fd)
    next(reader)

    mapping_types = {}
    for row in reader:
        if row["map_name_arches"] in computed_fields and row["map_name_arches"] not in fields_to_coalesce:
            continue
        if "/" in row["map_name_arches"]:
            field, _ = row["map_name_arches"].split("/", -1)
            assert "/" not in field, "Can only handle one level of grouping (semantic field)"
            assert \
                field in computed_fields, \
                f"Cannot automatically ingest semantic fields, you must create a computed field for {field}"
            continue
                
        typ = re.sub("\[(.*)\]", r"\1", row["type"])
        is_array = (typ != row["type"])
        typ = mapping_types_standardization.get(typ, typ) or "str"
       
        if typ == "concept":
            typ = row["map_name_arches"]
            mapping_cb = partial(concept_lookup, row["map_name_arches"])
        elif typ in types:
            mapping_cb = types[typ]
        else:
            print(f"Skipping unknown type: {typ} for {row['Name_hed']}")

        if is_array:
            typ = f"[{typ}]"
            fields_to_coalesce.setdefault(row["map_name_arches"], [])
            fields_to_coalesce[row["map_name_arches"]].append(row["Name_hed"])
            if row["map_name_arches"] not in computed_fields:
                computed_fields[row["map_name_arches"]] = (partial(_coalesce_to_array, target=row["map_name_arches"], cb=mapping_cb), typ)
            else:
                assert computed_fields[row["map_name_arches"]][1] == typ, f"{row['map_name_arches']} {typ}"
        else:
            mapping_types[row["map_name_arches"]] = (typ, mapping_cb)
    
    #mapping = {row["Name_hed"]: row["map_name_arches"] for row in reader if row["map_name_arches"] != ""}
    fd.seek(0)
    next(reader)
    
    mapping = {}
    for row in reader:
        if row["map_name_arches"] in ("monument_type", "monument_name"):
            continue
        if row["map_name_arches"] != "" and "/" not in row["map_name_arches"]:
            mapping[row["Name_hed"]] = row["map_name_arches"]

    #mapping_types = {row["map_name_arches"]: ((row.get("Type") or "str") if row["Type"] != "concept" else row["map_name_arches"], types.get(row.get("Type") or "str") if row["Type"] != "concept" else partial(types.get(row["Type"]), row["map_name_arches"])) for row in reader}

Skipping unknown type: upload for MainPictureFile
Skipping unknown type: upload? for GridRef


In [16]:
fields_to_coalesce

{'description': ['SiteSummary',
  'EditedType',
  'Condition',
  'SiteManagement',
  'WardenConditionRptSummary'],
 'external_cross_reference': ['B_FileNo', 'DANIFarmNo', 'OtherRef']}

In [17]:
computed_fields

{'monument_name': (<function __main__.monument_name(row)>, 'str'),
 'monument_type': (<function __main__.monument_type(row)>, '[MonumentType]'),
 'localities': (<function __main__.localities(row)>,
  '[MonumentLocalitiesInput]'),
 'system_reference_numbers': (<function __main__.system_reference_numbers(row)>,
  '[MonumentSystemReferenceNumbersInput]'),
 'description': (functools.partial(<function _coalesce_to_array at 0x7f3ff6804af0>, target='description', cb=<class 'str'>),
  '[str]'),
 'external_cross_reference': (functools.partial(<function _coalesce_to_array at 0x7f3ff6804af0>, target='external_cross_reference', cb=<class 'str'>),
  '[str]')}

In [18]:
df = pd.read_csv("monuments_merged.csv")
df['geometry'] = df['geometry'].apply(lambda r: wkt.loads(r) if pd.notna(r) else None)
df = gpd.GeoDataFrame(df, crs='epsg:29901')

/tmp/philtweir/ipykernel_13371/1783192268.py:1: DtypeWarning: Columns (26,33,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("monuments_merged.csv")


In [19]:
df[df["GenTypePossible"].notna()]

,MonID,County,SMRNo,Council,Town,Barony,Parish,Condition,SiteName,SiteSummary,...,DescripScheduledArea,AllTownlands,SiteManagement,DateGISUpdated,AllGenPeriods,Category,SMR_new,MONID,Located,geometry
7,8,Antrim,004:017,Causeway Coast And Glens,NaN,Cary,Ballintoy,No Visible Remains (All Above Ground Features ...,Carnaboghil,"At Carnaboghil quite high up, on a S-facing hi...",...,NaN,Lemnagh Beg,NaN,2009-08-19 16:39:28.580,Prehistoric,NaN,ANT004:017,8.0,Unlocated,POINT (302500.000 442500.000)
13,14,Antrim,004:023,Causeway Coast And Glens,NaN,Cary,Ballintoy,No Visible Remains (All Above Ground Features ...,NaN,"In White Park Bay, in undulating sand dunes & ...",...,NaN,White Park,NaN,2009-08-19 16:39:28.593,Uncertain; Prehistoric,NaN,ANT004:023,14.0,Unlocated,POINT (301000.000 443000.000)
83,84,Antrim,001:012,Causeway Coast And Glens,NaN,Cary,Rathlin Island,No Visible Remains (All Above Ground Features ...,NaN,Source? and authority? for this site as a cair...,...,NaN,Ballynagard,NaN,2009-08-19 16:39:40.517,Uncertain,NaN,ANT001:012,84.0,Located,POINT (314290.000 451930.000)
87,88,Antrim,001:016,Causeway Coast And Glens,NaN,Cary,Rathlin Island,Traces Only (No Definable Features),Carrickagile,Situated on the E facing shore of the island i...,...,NaN,Craigmacagan,NaN,2009-08-19 16:39:32.813,Uncertain,NaN,ANT001:016,88.0,Located,POINT (316000.000 450600.000)
90,91,Antrim,001:019,Causeway Coast And Glens,NaN,Cary,Rathlin Island,Well Preserved (Complete/Substantially Complete),Illandoey,In a bay on the W shore of Ushet Lough is a li...,...,NaN,Roonivoolin,NaN,2009-08-19 16:39:28.610,E.Christ.,NaN,ANT001:019,91.0,Located,POINT (314910.000 448450.000)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16669,16716,Armagh,026:054,Newry Mourne And Down,NaN,Orior Upper,Newry,Excavated,NaN,Archaeological remains uncovered during a road...,...,NaN,Derry Beg,NaN,NaN,Uncertain; Bronze Age,NaN,ARM026:054,16716.0,NaN,POINT (306994.834 328358.612)
16671,16718,Down,033:058,Armagh City Banbridge And Craigavon,NaN,Orior Lower,Ballymore,Well Preserved (Complete/Substantially Complete),NaN,The remains of a ditch like feature were uncov...,...,NaN,Federnagh,NaN,NaN,Post-Med,NaN,DOW033:058,16718.0,NaN,POINT (305930.780 339500.056)
16677,16724,Antrim,040:106,Mid And East Antrim,NaN,Belfast Lower,Larne,Substantial Remains (Vast Majority Definable),NaN,A feature previously identified in ANT 040:105...,...,NaN,Curran And Drumaliss,NaN,NaN,Uncertain,NaN,ANT040:106,16724.0,NaN,POINT (341140.289 402933.784)
16685,16732,Down,002:049,Ards And North Down,NaN,Ards Lower,Bangor,Excavated,NaN,A possible burnt mound consisting of two shall...,...,NaN,Balloo Lower,NaN,NaN,E.B.A.,NaN,DOW002:049,16732.0,NaN,POINT (355208.227 382634.347)


In [20]:
mapper = arches_client.Mapper(mapping, computed_fields, mapping_types, type_to_type, skip)

In [21]:
mapper.map_row(df.iloc[7])

{'MonID': 8,
 'County': 'Antrim',
 'SMRNo': '004:017',
 'Council': 'Causeway Coast And Glens',
 'Town': nan,
 'Barony': 'Cary',
 'Parish': 'Ballintoy',
 'Condition': 'No Visible Remains (All Above Ground Features Removed)',
 'SiteName': 'Carnaboghil',
 'SiteSummary': 'At Carnaboghil quite high up, on a S-facing hill slope, near the summit, with superb views SE-S-W. Designated "Carnaboghil" on the 1935 OS 6" map. There are now no visible remains of a cairn, the site of which could not be located. The father of the current owner stated that he had never heard of a cairn at this location. The most likely situation for such a monument is on top of the hill just N of the farm at the junction of this townland with Lemnagh Beg.',
 'Map12500': 'IG7',
 'Altitude': '0500+F',
 'GridRef': 'D0250042500',
 'Protection': nan,
 'EditedType': 'Cairn (Unlocated): Carnaboghil',
 'GenType': 'Cairn',
 'GenTypePossible': 'Possible',
 'DescribedBy': nan,
 'LastDescribed': '1993-08-25',
 'InputBy': nan,
 'Dat

# Send for processing

In [22]:
def make_client(mapping, computed_fields, mapping_types, rows, split=False):
    import arches_client
    import asyncio
    client = arches_client.ResourceClient(ENDPOINT, "monument", "monument_name")
    mapper = arches_client.Mapper(mapping, computed_fields, mapping_types, type_to_type, skip)
    
    client.connect(timeout=300)
    results = asyncio.run((mapper.bulk_create_monuments_from_split_df if split else mapper.bulk_create_monuments_from_rows)(client, rows))
    return results["bulkCreateMonument"]["monuments"][:20]

In [23]:
def install():
    import os
    os.system("pip install graphene gql aiohttp dateparser slugify 'bokeh<3'") 
dask.delayed(install)().compute()

In [24]:
import time

In [25]:
df[df["EditedType"].astype(str).str.contains("A.P. Site - Booley Huts & Field Boundaries")]

,MonID,County,SMRNo,Council,Town,Barony,Parish,Condition,SiteName,SiteSummary,...,DescripScheduledArea,AllTownlands,SiteManagement,DateGISUpdated,AllGenPeriods,Category,SMR_new,MONID,Located,geometry
999,1000,Antrim,010:010,Causeway Coast And Glens,NaN,Cary,Culfeightrin,Some Remains (Some Definable Features),NaN,Quite high up in an exposed location on a terr...,...,NaN,Cushleake Mountain North,NaN,2009-08-19 16:39:36.843,Uncertain,NaN,ANT010:010,1000.0,Located,POINT (323130.000 437920.000)


In [26]:
from tqdm import tqdm
import time
def bulk_create_from_split_df(split_df, start_batch=0):
    results = []
    #it = tqdm(total=len(df))
    df_length = len(split_df) * len(split_df[0])
    
    cluster = LocalCluster()
    with Client(cluster) as dask_client:
        dask_client.upload_file("arches_client.py")
        for batch, rows in enumerate(split_df[start_batch:]):
            batch += start_batch
            print(batch, rows.iloc[0]["SMR_new"], len(rows) * batch, "of", df_length)
            while True:
                try:
                    print("create", batch)
                    bulk_create = dask.delayed(make_client)(mapping, computed_fields, mapping_types, rows)
                    print("compute", batch)
                    bulk_create.compute()
                    print("complete", batch)
                except Exception:
                    print(f"Exception in batch {batch} from row {batch * len(rows)} to row {(batch + 1) * len(rows)}")
                    time.sleep(10)
                else:
                    break
            #new_results = [mon["id"] for mon in (await bulk_create_monuments_from_rows(rows))["bulkCreateMonument"]["monuments"]]
            #results += new_results
            #it.update(len(rows))
    return results

# Send a specific set of rows

In [27]:
mapper.map_row(df.iloc[1])

{'MonID': 2,
 'County': 'Antrim',
 'SMRNo': '004:011',
 'Council': 'Causeway Coast And Glens',
 'Town': nan,
 'Barony': 'Cary',
 'Parish': 'Ballintoy',
 'Condition': 'Some Remains (Some Definable Features)',
 'SiteName': 'Cloughnaboghil, Druids Stone, Cloghnaboghil, Cloghnaboghil',
 'SiteSummary': 'On top of an enimence on a steep hill slope. The remains of this megalithic tomb are almost completely overgrown, hindering survey. It consists of a circular cairn indicated by 4 kerb stones in an arc N-NE. Elsewhere the kerb is overgrown. A single chamber in the interior is formed by 4 upright slabs supporting a broken capstone which is 1.58m long x 1.12 wide x 0.5m thick. The entrance to the chamber, 0.6m wide, faces SE. Access to the chamber is not now possible & it is c.0.8m wide.',
 'Map12500': 'IG7-4',
 'Altitude': '0400+F',
 'GridRef': 'D0223043300',
 'Protection': 'Scheduled',
 'EditedType': 'Passage Tomb: Cloghaboghil',
 'GenType': 'Megalithic Tomb',
 'GenTypePossible': nan,
 'Descr

# Send a sample (<10s)

In [28]:
import numpy as np

In [29]:
segment = df
split_df = np.array_split(segment, (len(segment) - 1) // 20 + 1)

'lang': 'en', 'type': <class 'str'>, 'nodeid': '325a430a-efe4-11eb-810b-a87eeabdefba', 'nodegroupid': '325a2f2f-efe4-11eb-9b0c-a87eeabdefba'}
[{'325a430a-efe4-11eb-810b-a87eeabdefba': {'en': {'value': 'ANT004:029', 'direction': 'ltr'}}}] TILES
primary_reference_number 20
<class 'int'>
20 value
{'325a2f33-efe4-11eb-b0bb-a87eeabdefba': 20}
{'325a2f2f-efe4-11eb-9b0c-a87eeabdefba': [<Tile: Tile object (bfb3a4a3-4120-4640-8149-127ea82120c7)>]}
[<Tile: Tile object (a286b476-6160-434d-9fde-ed6114a62ded)>, <Tile: Tile object (29239c53-15c2-419e-8c7c-81702f277eeb)>, <Tile: Tile object (5a916b03-ae8e-4321-ba1d-bc3a9d4c59ef)>, <Tile: Tile object (0b5be174-c97a-4956-8859-3c18cd55778f)>, <Tile: Tile object (9fb7521e-df22-4d38-a37b-665e3ded16e5)>, <Tile: Tile object (4939755f-88d2-4d6d-8597-0a461d366227)>, <Tile: Tile object (de73c195-abc5-4142-b166-c9461c9a40e1)>, <Tile: Tile object (dd565d56-e459-48cb-a02f-1846876b29b8)>, <Tile: Tile object (0fbc02f1-219c-45f1-af9e-2ca30653b056)>, <Tile: Tile object (0394ad4b-32dc-4989-a02c-617552afc10e)>, <Tile: Tile object (8ca96094-c07b-4dc3-8c9c-ceb694ef7640)>, <Tile: Tile object (bfb3a4a3-4120-4640-8149-127ea82120c7)>]
[<Tile: Tile object (a286b476-6160-434d-9fde-ed6114a62ded)>, <Tile: Tile object (29239c53-15c2-419e-8c7c-81702f277eeb)>, <Tile: Tile object (5a916b03-ae8e-4321-ba1d-bc3a9d4c59ef)>, <Tile: Tile object (0b5be174-c97a-4956-8859-3c18cd55778f)>, <Tile: Tile object (9fb7521e-df22-4d38-a37b-665e3ded16e5)>, <Tile: Tile object (4939755f-88d2-4d6d-8597-0a461d366227)>, <Tile: Tile object (de73c195-abc5-4142-b166-c9461c9a40e1)>, <Tile: Tile object (dd565d56-e459-48cb-a02f-1846876b29b8)>, <Tile: Tile object (0fbc02f1-219c-45f1-af9e-2ca30653b056)>, <Tile: Tile object (0394ad4b-32dc-4989-a02c-617552afc10e)>, <Tile: Tile object (8ca96094-c07b-4dc3-8c9c-ceb694ef7640)>, <Tile: Tile object (bfb3a4a3-4120-4640-8149-127ea82120c7)>]
[{'87d3d7dc-f44f-11eb-bee9-a87eeabdefba': {'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-6.372118, 55.239588]}, 'properties': {}}], 'properties': {'nodeId': '87d3d7dc-f44f-11eb-bee9-a87eeabdefba'}}}, {'87d3d7c5-f44f-11eb-8459-a87eeabdefba': '90bfc04c-a535-4676-b591-263efe6daf40', '87d3c3ea-f44f-11eb-b532-a87eeabdefba': '1aecefd4-33f2-4d42-a39e-3b0936d1dc4c'}, {'87d3d7c5-f44f-11eb-8459-a87eeabdefba': 'c6844e5b-2f3a-455f-9dda-e6344eff2cd1', '87d3c3ea-f44f-11eb-b532-a87eeabdefba': 'a62c7549-b76a-41cf-a0de-b0fdbccd2591'}, {'87d3d7c5-f44f-11eb-8459-a87eeabdefba': 'fd9491ce-e5a2-4b52-b937-b52fb0595f5e', '87d3c3ea-f44f-11eb-b532-a87eeabdefba': '36790995-8ead-43e0-b96c-5989747c08c6'}, {'87d3d7c5-f44f-11eb-8459-a87eeabdefba': '31

In [30]:
df["SiteManagement"].notna().count()

16695

In [31]:
len(df)

16695

In [32]:
df['SMR_new'].iloc[0]

'ANT004:010'

In [33]:
df[pd.isna(df['GenType'])]

,MonID,County,SMRNo,Council,Town,Barony,Parish,Condition,SiteName,SiteSummary,...,DescripScheduledArea,AllTownlands,SiteManagement,DateGISUpdated,AllGenPeriods,Category,SMR_new,MONID,Located,geometry
16040,16045,Fermanagh,229:062,Fermanagh And Omagh,NaN,Tirkennedy,Derryvullan,Some Remains (Some Definable Features),NaN,"This site was reported by the landowner, who h...",...,NaN,Derrybeg,NaN,2009-08-19 16:39:37.783,Post-Med,NaN,FER229:062,16045.0,Located,POINT (225850.000 340690.000)
16050,16055,Armagh,012:121,Armagh City Banbridge And Craigavon,NaN,Armagh,Eglish,No Visible Remains (All Above Ground Features ...,NaN,Vertical aerial photographs taken in Nov. 1983...,...,NaN,Lisadian,NaN,2009-08-19 16:39:32.593,Uncertain,NaN,ARM012:121,16055.0,Located,POINT (284051.000 346241.000)
16053,16058,Armagh,012:131,Armagh City Banbridge And Craigavon,NaN,Armagh,Eglish,No Visible Remains (All Above Ground Features ...,NaN,"A dark, semi-circular vegetation mark, visible...",...,NaN,Tullylost,NaN,2009-08-19 16:39:32.593,Uncertain,NaN,ARM012:131,16058.0,Located,POINT (285198.000 344561.000)
16054,16059,Armagh,024:012,Newry Mourne And Down,NaN,Fews Upper,Newtownhamilton,NaN,NaN,"In notes dated Sept. 1950, Paterson wrote ""On ...",...,NaN,Tullyvallan,NaN,NaN,Uncertain,NaN,ARM024:012,16059.0,Located,POINT (290000.000 328000.000)
16095,16102,MRA,009:012,Causeway Coast And Glens,NaN,Cary,Culfeightrin,Substantial Remains (Vast Majority Definable),Tornaroan Salt Pan,This early saltpan is located on the shore to ...,...,The scheduled area is outlined in red on the a...,Tornaroan,NaN,2010-03-08 11:36:36.743,C18Th,Aquatic Monuments,MRA009:012,16102.0,Located,POINT (315100.000 441950.000)
16469,16489,Down,001:044,Ards And North Down,NaN,Castlereagh Lower,Bangor,Substantial Remains (Vast Majority Definable),Grey Point Fort,Situated on the southern shores of Belfast Lou...,...,NaN,Ballygrot,NaN,NaN,C20Th,NaN,DOW001:044,16489.0,Located,POINT (345762.000 383268.000)
16571,16606,Londonderry,002:503,Causeway Coast And Glens,NaN,Coleraine,Dunboe,Well Preserved (Complete/Substantially Complete),NaN,"This is one of a pair of railway tunnels, open...",...,The scheduled area is outlined in red on the a...,Downhill,NaN,NaN,C19Th,Standing Buildings,LDY002:503,16606.0,Located,POINT (276490.000 436240.000)
16617,16664,Fermanagh,192:096,Fermanagh And Omagh,NaN,Tirkennedy,Derrybrusk,NaN,NaN,Drumconnis(4). Burnt Mound. Concave Profile. 6...,...,NaN,Drumconnis,NaN,NaN,NaN,NaN,FER192:096,16664.0,NaN,POINT (225540.000 351010.000)
16680,16727,Antrim,052:159,Antrim And Newtownabbey,NaN,Carrickfergus,Parish Of Carrickfergus Or St. Nicholas,Excavated,NaN,"Exacvation at the site by Gahan & Long, under ...",...,NaN,West Division,NaN,NaN,M.B.A.; Uncertain,NaN,ANT052:159,16727.0,NaN,POINT (337003.000 384669.000)


In [34]:
16040 / 20

802.0

In [35]:
mapper.map_row(split_df[813].iloc[0])

{'MonID': 16267,
 'County': 'MRA',
 'SMRNo': '003:162',
 'Council': 'Causeway Coast And Glens',
 'Town': nan,
 'Barony': 'Cary',
 'Parish': 'Rathlin Island',
 'Condition': 'Substantial Remains (Vast Majority Definable)',
 'SiteName': nan,
 'SiteSummary': 'Situated above the stony beach, between a bedrock outcrop and a grass-covered scarp at Portantonnish, the boat shelter lies c. 12m to the NE of MRA 3:161, and utilises the same bedrock outcrop as that structure. The site is sheltered from WNW and ESE, and is orientated NNE/SSW. It comprises a semi-circular setting of drystone wall, using the bedrock as an enclosing element to the W. The stones are undressed, sub-angular boulders, and the interior is overgrown, with some collapse evident. The wall is 30cm in width, 81cm (1-3 courses) in maximum height and encloses an area 3.7m by 1.95m. There is no obvious entrance. The structure was probably used as a boat and/or gear shelter. See also MRA 3:60 and 3:161.',
 'Map12500': 'CS 1',
 'Alti

In [36]:
import datetime
segment = df
segment['_batch'] = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
batch_used = False
split_df = np.array_split(segment, (len(segment) - 1) // 20 + 1)

In [37]:
row = df[df["SMR_new"].str.contains("ANT004:010")]
mapper.apply(row.iloc[0])

{'description': ['This site consists of a sub-circular mound 19.1m N-S x 24.3m E-W at the base, 6m x 9m on the top, 5.2m high at N, reduced to 2.8m at S. At least 15 slabs & boulders embedded in the mound c.1.5m below the top could form a cairn kerb. Only one stands to any height, a triangular slab 0.93m high at NW. The site was excavated by Grey in 1870, when he found the remains of a skeleton on "a rude platform of stones", which he considered a secondary burial.',
  'Cairn',
  'Substantial Remains (Vast Majority Definable)',
  '18/02/2009 While there is a trail through the dunes leading to the cairn the site itself has good cover of short grass which is grazed occassionally by cows. The stones are undisturbed.   11/08/2006 Area is covered with rough grasses and meadow-type wild flowers/species. The stones are un-disturbed and there are well-worn tracks up to the hillock indicating visitor usage. No thhreat.  24/08/2000 Condition is very good. Short grass covers scheduled area and st

In [1]:
if batch_used:
    print("WARNING: BATCH HAS BEEN USED")
    #time.sleep(5)
batch_used = True
bulk_create_from_split_df(split_df[0:1], 0)

NameError: name 'batch_used' is not defined

# Send entire dataset (>1h)

In [133]:
segment = df
split_df = np.array_split(segment, (len(segment) - 1) // 20 + 1)

This will return immediately.

In [357]:
dask.distributed.fire_and_forget(dask_client.submit(make_client, mapping, computed_fields, mapping_types, split_df, split=True))